In [1]:
from agent_framework import MCPStdioTool, MCPStreamableHTTPTool, MCPStdioTool


In [2]:
tool = MCPStreamableHTTPTool(
            name="Microsoft Learn MCP",
            url="https://learn.microsoft.com/api/mcp",
            # we don't require approval for microsoft_docs_search tool calls
            # but we do for any other tool
            # approval_mode={"never_require_approval": ["microsoft_docs_search"]},
        )

In [3]:
tool2 = MCPStdioTool(  
    name="filesystem",  
    command="npx",  
    args=["-y", "@modelcontextprotocol/server-filesystem", r"C:\Users\blain\Documents"],  
    description="File system operations",  
)

In [4]:
tool3 = MCPStreamableHTTPTool(
            name="localhost MCP",
            url="http://localhost:1337/mcp",
        )


In [ ]:
import logging
import httpx
from httpx._content import ByteStream

# Restore any previous request patch from earlier runs
if "original_request" in globals():
    httpx.AsyncClient.request = original_request
    del globals()["original_request"]

# Reset logging handlers so we control formatting
root_logger = logging.getLogger()
for handler in list(root_logger.handlers):
    root_logger.removeHandler(handler)
logging.basicConfig(level=logging.DEBUG, format='\n[%(name)s - %(funcName)s]: %(message)s')
# Ensure httpx logs adopt the simplified format
httpx_logger = logging.getLogger("httpx")
httpx_logger.handlers.clear()
httpx_logger.setLevel(logging.DEBUG)
httpx_logger.propagate = True


# Also enable openai library logging if available
# logging.getLogger("openai").setLevel(logging.DEBUG)

if not hasattr(httpx.AsyncClient, "_original_send_single_request"):
    httpx.AsyncClient._original_send_single_request = httpx.AsyncClient._send_single_request

async def logged_send_single_request(self, request):
    logger = logging.getLogger("httpx")
    if logger.isEnabledFor(logging.DEBUG):
        stream = request.stream
        if isinstance(stream, ByteStream):
            body = stream._stream
            if body:
                try:
                    logger.debug("Request body (utf-8): %s", body.decode("utf-8"))
                except UnicodeDecodeError:
                    logger.debug("Request body (bytes): %s", body)
            else:
                logger.debug("Request body: <empty>")
        else:
            logger.debug("Request body not logged (stream type: %s)", type(stream).__name__)
    response = await httpx.AsyncClient._original_send_single_request(self, request)
    if logger.isEnabledFor(logging.DEBUG):
        # logger.debug("Response headers: %s", dict(response.headers))
        logger.debug(f"Response {response.status_code}: {await response.aread()}")
    return response

if not getattr(httpx.AsyncClient, "_logging_patch_applied", False):
    httpx.AsyncClient._send_single_request = logged_send_single_request
    httpx.AsyncClient._logging_patch_applied = True

from agent_framework.openai import OpenAIChatClient

# tool = MCPStreamableHTTPTool(
#             name="localhost MCP",
#             url="http://localhost:1337/mcp",
#         )
llm = OpenAIChatClient(
        api_key="ollama", # Just a placeholder, Ollama doesn't require API key
        # base_url="http://localhost:11434/v1",
        base_url="http://ollama.home/v1",
        model_id="gpt-oss:20b",
    )
agent = llm.create_agent(
        name="test_agent",
        instructions="You are a helpful agent. You use Model Context Protocol (MCP) tools to answer user questions. "\
            "You can only respond using the tools available to you. Do not make up tool functionality. The tools will be" \
                "Provided to you in the prompt.",
        tools=[tool3],
    )

query = "hello."
print(f"User: {query}")
print("\n=== HTTP Request/Response Details Below ===\n")
result = await agent.run(query)
print(f"\n=== End of HTTP Details ===\n")
print(f"Agent: {result}\n")



[openai._base_client _build_request]: Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'idempotency_key': 'stainless-python-retry-d5bf08d9-86bf-4376-8c2f-9d9982779208', 'json_data': {'messages': [{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a helpful agent. You use Model Context Protocol (MCP) tools to answer user questions. You can only respond using the tools available to you. Do not make up tool functionality. The tools will beProvided to you in the prompt.'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': 'hello.'}]}], 'model': 'gpt-oss:20b', 'stream': False, 'tool_choice': 'auto', 'tools': [{'type': 'function', 'function': {'name': 'speech_speak_to_users', 'description': 'Speaks to users using text-to-speech. Returns base64-encoded audio data.', 'parameters': {'properties': {'speech': {'title': 'Speech', 'type': 'string'}}, 'required': ['speech'], 'title': 'speech_speak_to_users_input', 'type': 'object'}}}, {'type': 'func

User: hello.

=== HTTP Request/Response Details Below ===




[httpcore.connection atrace]: connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x0000018A86B06060>

[httpcore.http11 atrace]: send_request_headers.started request=<Request [b'POST']>

[httpcore.http11 atrace]: send_request_headers.complete

[httpcore.http11 atrace]: send_request_body.started request=<Request [b'POST']>

[httpcore.http11 atrace]: send_request_body.complete

[httpcore.http11 atrace]: receive_response_headers.started request=<Request [b'POST']>

[httpcore.http11 atrace]: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Server', b'openresty'), (b'Date', b'Tue, 04 Nov 2025 02:37:59 GMT'), (b'Content-Type', b'application/json'), (b'Content-Length', b'468'), (b'Connection', b'keep-alive'), (b'X-Served-By', b'ollama.home')])

[httpx _send_single_request]: HTTP Request: POST http://ollama.home/v1/chat/completions "HTTP/1.1 200 OK"

[httpcore.http11 atrace]: receive_response_body.started request=<Request [b'POST']>

[


=== End of HTTP Details ===

Agent: Hello! How can I help you today?



In [6]:
import asyncio
import os
from random import randint
from typing import Annotated

from agent_framework.openai import OpenAIChatClient

"""
Ollama with OpenAI Chat Client Example

This sample demonstrates using Ollama models through OpenAI Chat Client by
configuring the base URL to point to your local Ollama server for local AI inference.
Ollama allows you to run large language models locally on your machine.

Environment Variables:
- OLLAMA_ENDPOINT: The base URL for your Ollama server (e.g., "http://localhost:11434/v1/v1/")
- OLLAMA_MODEL: The model name to use (e.g., "mistral", "llama3.2", "phi3")
"""


def get_weather(
    location: Annotated[str, "The location to get the weather for."],
) -> str:
    """Get the weather for a given location."""
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."


async def non_streaming_example() -> None:
    """Example of non-streaming response (get the complete result at once)."""
    print("=== Non-streaming Response Example ===")

    agent = llm.create_agent(
        name="WeatherAgent",
        instructions="You are a helpful weather agent.",
        tools=get_weather,
    )

    query = "What's the weather like in Seattle?"
    print(f"User: {query}")
    result = await agent.run(query)
    print(f"Agent: {result}\n")


async def streaming_example() -> None:
    """Example of streaming response (get results as they are generated)."""
    print("=== Streaming Response Example ===")

    agent = llm.create_agent(
        name="WeatherAgent",
        instructions="You are a helpful weather agent.",
        tools=get_weather,
    )

    query = "What's the weather like in Portland?"
    print(f"User: {query}")
    print("Agent: ", end="", flush=True)
    async for chunk in agent.run_stream(query):
        if chunk.text:
            print(chunk.text, end="", flush=True)
    print("\n")

In [7]:
await streaming_example()

=== Streaming Response Example ===
User: What's the weather like in Portland?
Agent: 


openai._base_client _build_request: Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'idempotency_key': 'stainless-python-retry-48e07892-ae1a-4a54-b4ad-faed25e2f4f7', 'json_data': {'messages': [{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a helpful weather agent.'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': "What's the weather like in Portland?"}]}], 'model': 'gpt-oss:20b', 'stream': True, 'stream_options': {'include_usage': True}, 'tool_choice': 'auto', 'tools': [{'type': 'function', 'function': {'name': 'get_weather', 'description': 'Get the weather for a given location.', 'parameters': {'properties': {'location': {'description': 'The location to get the weather for.', 'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weather_input', 'type': 'object'}}}]}}

openai._base_client request: Sending HTTP Request: POST http://ollama.home/v1/chat/completions

httpx logged_send_single_request: Requ

In Portland today, you’ll enjoy a bright, sunny day with temperatures reaching up to about 20 °C. It’s a nice time for outdoor activities—just remember to stay hydrated if you’re planning to be outside!

